### Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import nltk
from sklearn.preprocessing import StandardScaler

Load Data

In [2]:
df = pd.read_csv("Car_Reviews_Database.csv", delimiter=";", index_col=0, encoding='latin1')

In [3]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [4]:
df.head()

,Model,Review
Year,,
2009,Honda,Although arguably the first-generation Insight...
2009,Honda,2009 Honda Accord EX-L 4 : This car is very c...
2010,Honda,I have owed and driven Honda products for 20 y...
2010,Honda,Honda Accord Euro L : The seats are average bu...
2011,Honda,Honda HR-V: Continuous variable transmission ...


In [5]:
df.describe

<bound method NDFrame.describe of                                                    Model   \
Year                                                        
2009                                                Honda   
2009                                                Honda   
2010                                                Honda   
2010                                                Honda   
2011                                                Honda   
...                                                   ...   
NaN                                                   NaN   
NaN                                                   NaN   
NaN                                                   NaN   
NaN                                                   NaN   
;;; it's by far one of the most reliable cars I...    NaN   

                                                                                               Review  
Year                                                                

Jumlah nilai yang null

In [6]:
total = df.isnull().sum().sort_values(ascending=False)

percent = (df.isnull().sum()/df.isnull().count())\
.sort_values(ascending=False)

missing_data = pd.concat([total, percent], axis=1, \
                        keys=['Total', 'Missing Percent'])

missing_data['Missing Percent'] = \
missing_data['Missing Percent'].apply(lambda x: x * 100)

missing_data.loc[missing_data['Missing Percent'] > 10][:10]

,Total,Missing Percent
Review,193,47.772277
Model,192,47.524752


In [7]:
df.isnull().sum()

Model     192
Review    193
dtype: int64

### Data Cleaning

In [8]:
dc = pd.read_csv("Car_Reviews_Database.csv", delimiter=";", index_col=0, encoding='latin1')
dc = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [9]:
print(dc.shape)

(404, 2)


Drop Missing Value

In [10]:
dc = dc.dropna()
print(dc.shape)

(211, 2)


In [11]:
total = dc.isnull().sum().sort_values(ascending=False)

percent = (dc.isnull().sum()/dc.isnull().count())\
.sort_values(ascending=False)

missing_data = pd.concat([total, percent], axis=1, \
                        keys=['Total', 'Missing Percent'])

missing_data['Missing Percent'] = \
missing_data['Missing Percent'].apply(lambda x: x * 100)

missing_data.loc[missing_data['Missing Percent'] > 10][:10]

,Total,Missing Percent


In [12]:
dc.isnull().sum()

Model     0
Review    0
dtype: int64

In [13]:
dc.head(10)

,Model,Review
Year,,
2009,Honda,Although arguably the first-generation Insight...
2009,Honda,2009 Honda Accord EX-L 4 : This car is very c...
2010,Honda,I have owed and driven Honda products for 20 y...
2010,Honda,Honda Accord Euro L : The seats are average bu...
2011,Honda,Honda HR-V: Continuous variable transmission ...
2011,Honda,Not much has changed with the historically sec...
2012,Honda,Honda Ballade 150 1.5: This is the most reliab...
2012,Honda,Ride quality is top-notch though communication...
2013,Honda,Honda Jazz Hybrid 1.4 : This is my second Hond...


## Modelling

In [14]:
data_train = dc
data_test = dc

In [15]:
data_train.head()

,Model,Review
Year,,
2009,Honda,Although arguably the first-generation Insight...
2009,Honda,2009 Honda Accord EX-L 4 : This car is very c...
2010,Honda,I have owed and driven Honda products for 20 y...
2010,Honda,Honda Accord Euro L : The seats are average bu...
2011,Honda,Honda HR-V: Continuous variable transmission ...


In [16]:
data_test.head()

,Model,Review
Year,,
2009,Honda,Although arguably the first-generation Insight...
2009,Honda,2009 Honda Accord EX-L 4 : This car is very c...
2010,Honda,I have owed and driven Honda products for 20 y...
2010,Honda,Honda Accord Euro L : The seats are average bu...
2011,Honda,Honda HR-V: Continuous variable transmission ...


In [17]:
# Case Folding
data_train['Review'] = data_train['Review'].str.lower()

In [18]:
# Tokenization Train
from nltk.tokenize import RegexpTokenizer
train_token = RegexpTokenizer(data_train['Review'])
print(train_token)

RegexpTokenizer(pattern=Year
2009    although arguably the first-generation insight...
2009    2009 honda accord ex-l 4  : this car is very c...
2010    i have owed and driven honda products for 20 y...
2010    honda accord euro l : the seats are average bu...
2011     honda hr-v: continuous variable transmission ...
                              ...                        
2019    honda civic ex: very clever design and very sp...
2019    hyundai sonata 2.4: large car and spacious car...
2019    if you can deal with (or prefer?) rear-wheel d...
2019     ford escape xlt 2.5 : i really didn't want to...
2019    the bmw 3 series is known as a driver's car an...
Name: Review, Length: 211, dtype: object, gaps=False, discard_empty=True, flags=re.UNICODE|re.MULTILINE|re.DOTALL)


In [19]:
import re, string
from nltk.tokenize import word_tokenize

def preprocessText(text, remove_stops=False):
    
    # Remove Everything Between Hard Brackets
    text = re.sub(pattern="\[.+?\]( )?", repl='', string=text)

    # Change "word+'" to "word+ng"
    text = re.sub(pattern="n\\\' ", repl='ng ', string=text)

    # Remove x4 and (x4)
    text = re.sub(pattern="(\()?x\d+(\))?", repl=' ', string=text)

    # Fix Apostrophe Issues
    text= re.sub(pattern="\\x91", repl="'", string=text)
    text = re.sub(pattern="\\x92", repl="'", string=text)
    text= re.sub(pattern="<u\+0092>", repl="'", string=text)
        
    # Special Cases
    text = re.sub(pattern="'til", repl="til", string=text)
    text = re.sub(pattern="'til", repl="til", string=text)
    text = re.sub(pattern="gon'", repl="gon", string=text)

    # Remove \n from Beginning
    text = re.sub(pattern='^\n', repl='', string=text)
    
    # Remove Double Spaces and Beginning with Whitespace
    text = re.sub(pattern='( ){2,}', repl=' ', string=text)
    text = text.strip()

    # Strip , ! ?, : and Remaining \n from Review
    text = ''.join([char.strip(",!?:") for char in text])
    text = text.replace('\n', ' ')
  

    #---------------------- Text Expansion ---------------------------------
    
    # specific
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"won\’t", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"can\’t", "can not", text)
    text = re.sub(r"let's", "let us", text)
    text = re.sub(r"let\’s", "let us", text)
    text = re.sub(r"ain't", "aint", text)
    text = re.sub(r"ain\’t", "aint", text)
    text = re.sub(r"wanna", "want to", text)
    text = re.sub(r"gonna", "going to", text)
    text = re.sub(r"gotta", "go to", text)
    
    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"n\’t", " not", text)
    text = re.sub(r"\’re", " are", text)
    text = re.sub(r"\’s", " is", text)
    text = re.sub(r"\’d", " would", text)
    text = re.sub(r"\’ll", " will", text)
    text = re.sub(r"\’t", " not", text)
    text = re.sub(r"\’ve", " have", text)
    text = re.sub(r"\’m", " am", text)
    text = re.sub(r"\â", "a", text)
    text = re.sub(r"sunshineas", "sunshine as", text)
    
    # Remove Symbols
    text = re.sub(r"[^\w\s]","", text)
    
    # Remove Single Char
    text = re.sub(r"\b[a-zA-Z]\b","", text)
   
    #Remove Number
    text = re.sub(r"\d+", "", text)
    
    #Remove Duplicate Char
    text = re.sub(r"o+h", "oh", text)
    text = re.sub(r"oh+", "", text)
    
    # Remove Remaining Punctuation
    punc = string.punctuation
    text = ''.join([char for char in text if char not in punc])

    # Remove Stopwords
    if remove_stops:
        stops = stopwords.words('english')
        data_train['Review'] = ' '.join([word for word in data_train['Review'].split(' ') if word not in stops])
    return(text)

data_train['Review'] = data_train['Review'].apply(preprocessText)

# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_train['Review'] = data_train['Review'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data_train['Review'])
print('\n\n\n')


Tokenizing Result : 

Year
2009    [although, arguably, the, firstgeneration, ins...
2009    [honda, accord, exl, this, car, is, very, comf...
2010    [have, owed, and, driven, honda, products, for...
2010    [honda, accord, euro, the, seats, are, average...
2011    [honda, hrv, continuous, variable, transmissio...
                              ...                        
2019    [honda, civic, ex, very, clever, design, and, ...
2019    [hyundai, sonata, large, car, and, spacious, c...
2019    [if, you, can, deal, with, or, prefer, rearwhe...
2019    [ford, escape, xlt, really, did, not, want, to...
2019    [the, bmw, series, is, known, as, driver, is, ...
Name: Review, Length: 211, dtype: object






In [20]:
# Stopwords Removal
from nltk.corpus import stopwords

stop = stopwords.words('english')
data_train['Review'] = data_train['Review'].apply(lambda x: [item for item in x if item not in stop])

In [21]:
# Stemming
from nltk.stem.porter import PorterStemmer

stemming = PorterStemmer()
data_train['Review'] = data_train['Review'].apply(lambda x: [stemming.stem(y) for y in x])

In [22]:
# Lemmatization
from nltk.stem import WordNetLemmatizer

lemmatization = WordNetLemmatizer()
data_train['Review'] = data_train['Review'].apply(lambda x: [lemmatization.lemmatize(y) for y in x])

In [23]:
data_train.head()

,Model,Review
Year,,
2009,Honda,"[although, arguabl, firstgener, insight, hybri..."
2009,Honda,"[honda, accord, exl, car, comfort, sporti, cyl..."
2010,Honda,"[owe, driven, honda, product, year, purchas, v..."
2010,Honda,"[honda, accord, euro, seat, averag, littl, rea..."
2011,Honda,"[honda, hrv, continu, variabl, transmiss, fail..."


In [24]:
print(data_train)

       Model                                              Review
Year                                                            
2009    Honda  [although, arguabl, firstgener, insight, hybri...
2009    Honda  [honda, accord, exl, car, comfort, sporti, cyl...
2010    Honda  [owe, driven, honda, product, year, purchas, v...
2010    Honda  [honda, accord, euro, seat, averag, littl, rea...
2011    Honda  [honda, hrv, continu, variabl, transmiss, fail...
...       ...                                                ...
2019   Honda   [honda, civic, ex, clever, design, spaciou, ca...
2019  Hyundai  [hyundai, sonata, larg, car, spaciou, car, ple...
2019  Genesis  [deal, prefer, rearwheel, drive, amaz, vehicl,...
2019     Ford  [ford, escap, xlt, realli, want, like, car, pu...
2019      BMW  [bmw, seri, known, driver, car, redesign, seri...

[211 rows x 2 columns]
